# Étude d'une équation de convection scalaire

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
#
import pyfvm.mesh  as mesh
import pyfvm.model as model
import pyfvm.field as data
from pyfvm.xnum        import *
from pyfvm.integration import *

On cherche à résoudre l'évolution instationnaire du problème linéaire de convection suivant

$$ \frac{\partial q}{\partial t} + a \frac{\partial q}{\partial x} = 0 $$

pour la quantité transportée $q(x,t)$ et la condition intiale $q_0(x)$ sur le domaine $[0;\ell]$ avec des conditions périodiques. On choisit $\ell=1\rm~m$ et $a=1\rm~m/s$. 


## Définition des maillages et du modèle physique


In [ ]:
mesh200 = mesh.unimesh(ncell=200, length=1.)
mesh100 = mesh.unimesh(ncell=100, length=1.)
mesh50  = mesh.unimesh(ncell=50,  length=1.)

mymodel     = model.convmodel(1.)

## Définition des solutions initiales

In [ ]:
# sinus packet
def init_sinpack(mesh):
    return np.sin(2*2*np.pi/mesh.length*mesh.centers())*(1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2        
    
# periodic wave
def init_sinper(mesh):
    k = 2 # nombre d'onde
    return np.sin(2*k*np.pi/mesh.length*mesh.centers())
    
# square signal
def init_square(mesh):
    return (1+np.sign(-(mesh.centers()/mesh.length-.25)*(mesh.centers()/mesh.length-.75)))/2

fig, ax = plt.subplots(1, 3, figsize=(15,4), sharey='row') ; ax[0].axes.set_ylim(-1.1, 1.1)
x = mesh200.centers()
ax[0].plot(x, init_sinper(mesh200))
ax[1].plot(x, init_sinpack(mesh200))
ax[2].plot(x, init_square(mesh200))

In [ ]:
endtime = 4   # final physical time of simulation
ntime   = 2   # number of intermediate snapshots
tsave   = np.linspace(0, endtime, num=ntime+1) 

initm = init_sinper
mesh  = mesh100
cfl   = 0.5
xnum  = extrapol1() # extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapol(1./3.) 
tnum  = explicit    # explicit, rk2, rk3ssp, rk4, implicit, trapezoidal=cranknicolson
#
field0          = data.scafield(mymodel, bc='p', nelem=mesh.ncell)   # intial field object
field0.qdata[0] = initm(mesh)                          # initialization
integrator = tnum(mesh, xnum)                          # define solver and integrator
#
# COMPUTATION
start = time.clock()
results = integrator.solve(field0, cfl, tsave)
print "cpu time (",integrator.nit,"it) :",time.clock()-start,"s"
#
# display results
style=['o', 'x', 'D', '*', 'o', 'o']
fig=plt.figure(1, figsize=(14,10))
plt.plot(mesh.centers(), results[0].qdata[0], '-')
labels = ["initial condition"]
for t in range(1,len(tsave)):
    plt.plot(mesh.centers(), results[t].qdata[0], style[t])
    labels.append(", t=%.1f"%results[t].time)
plt.legend(labels, loc='upper left',prop={'size':12})  


---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [ ]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style